In [ ]:
import os
import sys
from random import randint


module_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))  
if module_path not in sys.path:       
    sys.path.append(module_path)

In [ ]:
import librosa
import numpy as np
import tensorflow as tf
import IPython.display as ipd
from utils.helper import wav_to_spectrogram_clips, rebuild_audio_from_spectro_clips
from utils.dataset import create_samples

In [ ]:
samples = create_samples('Test')
test_sample = samples[randint(0, len(samples)-1)]

test_song = test_sample['mix']
vocal_track = test_sample['vocals']
ipd.Audio(vocal_track, rate=44100)

In [ ]:
sorted(os.listdir('../saved_model'))

In [ ]:
# load model
separator = tf.keras.models.load_model(os.path.join('../saved_model', 'conv_denoising_unet?time=20200226_1546.h5'))

In [ ]:
pred = separator.predict(wav_to_spectrogram_clips(test_song))

In [ ]:
pred_vocal = np.squeeze(pred[0], axis=-1)
print(pred_vocal.shape)

In [ ]:
separated_vocals = rebuild_audio_from_spectro_clips(pred_vocal)
ipd.Audio(separated_vocals, rate=44100)